In [10]:
import pandas as pd
import numpy as np
import json
import os
from collections import defaultdict

In [16]:
baselines = ["vanilla", "cot", "socratic", "subq"]
datasets = ["massmaps", "supernova", "politeness", "emotion", "cholec", "cardiac", "sepsis"]
models = ["gpt-4o"]

In [24]:
# Map for pretty LaTeX-style formatting of baselines
baseline_names = {
    "vanilla": "\\textbf{Vanilla}",
    "cot": "\\textbf{Chain-of-Thought}",
    "socratic": "\\textbf{Socratic Prompting}",
    "subq": "\\textbf{SubQ Decomposition}"
}

# Collect scores
results = defaultdict(list)

for baseline in baselines:
    for dataset in datasets:
        path = os.path.join(baseline, f"{dataset}_{model}.json")
        if not os.path.exists(path):
            print(f"File not found: {path}")
            continue
        with open(path, 'r') as f:
            try:
                data = json.load(f)
                scores = [entry["final_alignment_score"] for entry in data if "final_alignment_score" in entry and entry["final_alignment_score"] is not None]
                # If no scores found, check for the baseline key
                if(len(scores) == 0):
                    data = data[baseline]
                    scores = [entry["final_alignment_score"] for entry in data if "final_alignment_score" in entry and entry["final_alignment_score"] is not None]
                if scores:
                    avg = sum(scores) / len(scores)
                    results[(baseline, dataset)] = avg
            except Exception as e:
                print(scores)
                print(f"Error processing {path}: {e}")

# Generate LaTeX rows
print("Generating LaTeX table rows:\n")
for baseline in baselines:
    row = [baseline_names[baseline]]
    for dataset in datasets:
        avg_score = results.get((baseline, dataset), "")
        if isinstance(avg_score, float):
            row.append(f"{avg_score:.2f}")
        else:
            row.append("")  # Leave blank if not available
            print(f"Warning: No score found for {baseline} on {dataset}")
    print(" & ".join(row) + " \\\\")

Generating LaTeX table rows:

\textbf{Vanilla} & 0.42 & 0.83 & 0.68 & 0.50 & 0.30 & 0.47 & 0.54 \\
\textbf{Chain-of-Thought} & 0.44 & 0.81 & 0.64 & 0.59 & 0.34 & 0.50 & 0.53 \\
\textbf{Socratic Prompting} & 0.47 & 0.80 & 0.37 & 0.38 & 0.37 & 0.51 & 0.54 \\
\textbf{SubQ Decomposition} & 0.40 & 0.82 & 0.41 & 0.29 & 0.36 & 0.39 & 0.56 \\
